# Understanding Memory in LLMs

이전 노트북에서는 OpenAI 모델이 Azure AI 검색 쿼리의 결과를 향상시키는 방법을 성공적으로 살펴봤습니다. 
하지만 LLM과 대화에 참여하는 방법은 아직 발견하지 못했습니다. 예를 들어, [Bing Chat](http://chat.bing.com/)을 사용하면 이전 응답을 이해하고 참조할 수 있기 때문에 이것이 가능합니다.
LLM(대규모 언어 모델)에 메모리가 있다는 일반적인 오해가 있습니다. 이는 사실이 아닙니다. 지식을 보유하고 있기는 하지만 이전에 질문했던 정보를 기억하지는 못합니다.
이 노트북의 목표는 프롬프트와 문맥을 활용하여 LLM에 효과적으로 "메모리를 부여"하는 방법을 설명하는 것입니다.

In [ ]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
아주 간단한 예제를 사용하여 Azure OpenAI의 GPT 모델에 메모리가 있는지 확인해 보겠습니다. 다시 langchain을 사용하여 코드를 간소화하겠습니다.

In [ ]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [ ]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [ ]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [ ]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

In [ ]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [ ]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [ ]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [ ]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

Langchain 웹사이트에서 발췌:
    
메모리 시스템은 읽기와 쓰기라는 두 가지 기본 작업을 지원해야 합니다. 모든 체인은 특정 입력을 기대하는 몇 가지 핵심 실행 로직을 정의한다는 점을 기억하세요. 이러한 입력 중 일부는 사용자가 직접 제공하지만, 일부는 메모리에서 제공될 수도 있습니다. 체인은 주어진 실행에서 메모리 시스템과 두 번 상호 작용합니다.

    초기 사용자 입력을 받은 후 코어 로직을 실행하기 전에 체인은 메모리 시스템에서 읽고 사용자 입력을 보강합니다.
    핵심 로직을 실행한 후 답을 반환하기 전에 체인은 현재 실행의 입력과 출력을 메모리에 기록하여 향후 실행에서 참조할 수 있도록 합니다.
    
따라서 이 프로세스는 응답에 지연을 추가하지만 필요한 지연입니다 :)

![image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [ ]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-books"
indexes = [index1_name, index2_name]

In [ ]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)


prompts.py를 자세히 살펴보면 `DOCSEARCH_PROMPT`에 `history`라는 선택적 변수가 있습니다. 이제 이 변수를 사용할 때입니다. 기본적으로 프롬프트에 대화를 삽입하여 LLM이 응답하기 전에 이를 인식할 수 있도록 하는 플레이스 홀더입니다.

**Now let's add memory to it:**

In [ ]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [ ]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [ ]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

In [ ]:
config

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [ ]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

In [ ]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

In [ ]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [ ]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [ ]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [ ]:
config

In [ ]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

In [ ]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

In [ ]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

In [ ]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

In [ ]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

In [ ]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4-Turbo** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.